In [ ]:
import pandas as pd
import io
import os
import sqlite3
from google.colab import drive

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
subset = True

# Getting Full Data


*   Connecting with database (conn object)
*   Setting up database with tables specified in db.sql file
*   Importing datasets from folder
*   Creating OriginalDataSet table with all the data
*   To use a subset, set the subset flag to true


In [ ]:
# Mounting Drive and getting datasets directory and sql file

drive.mount("/content/drive/")
ds_dir = "/content/drive/My Drive/Facul/IBD/TP2/datasets"
db_file = "/content/drive/My Drive/Facul/IBD/TP2/db.sql"

MessageError: ignored

In [ ]:
# Reading DB definition file

conn = sqlite3.connect('/tmp/consult.db')
cursor = conn.cursor()

f = io.open(db_file, 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

In [ ]:
# Get names of dataset files

ds_files = os.listdir(ds_dir)

In [ ]:
# Read one file 
def read_ds_to_df(file_name):
  ano = int(file_name[:-4].split("_")[0])
  mes = int(file_name[:-4].split("_")[1])
  area = file_name[:-4].split("_")[3]

  skip = 7 if mes<9 else 3

  df = pd.read_csv('/'.join((ds_dir, file_name)), sep=';', header=None, skiprows=skip)
  df = df if mes<9 else df.drop(47,axis='columns')
  df.columns = [ 
      "estado", "bacia", "nome_poco_anp", "nome_poco_operador", "campo", "operador", "numero_contrato", "periodo", "oleo", "condensado", "petroleo", 
      "gas_natural_associado", "gas_natural_nao_associado", "gas_natural_total", "volume_gas_royalties", "agua", "instalacao_destino", "tipo_instalacao", 
      "tempo_producao", "periodo_da_carga", "corrente", "grau_api", "fp_lc", "fp_lv", "fp_mc", "fp_mv", "fp_pc", "fp_pv", 
      "metano", "etano", "propano", "isobutano", "butano", "isopentano", "npentano", "hexanos", "heptanos", "octanos", "nonanos", "decanos", 
      "undecanos", "oxigenio", "nitrogenio", "gas_carbonico", "densidade_glp_gas", "densidade_glp_liquido", "pcs_gp"
  ]
  df["ano"] = file_name[:-4].split("_")[0]
  df["mes"] = file_name[:-4].split("_")[1]
  df["area"] = file_name[:-4].split("_")[3]

  return df

In [ ]:
conn.execute('DELETE FROM OriginalDataSet;')

if subset:
  df_data = read_ds_to_df(ds_files[0])
  df_data.to_sql(name='OriginalDataSet', con=conn, if_exists='append', index=False)
else:
  for i in range(len(ds_files)):
    df_data = read_ds_to_df(ds_files[i])
    df_data.to_sql(name='OriginalDataSet', con=conn, if_exists='append', index=False)

# Locais

In [ ]:
conn.execute('DELETE FROM Locais;')

cmd = """
INSERT INTO
  Locais(estado, bacia, campo)
SELECT DISTINCT
  estado, bacia, campo
FROM 
  OriginalDataSet
"""

conn.execute(cmd)

In [ ]:
query = """
SELECT *
FROM 
  Locais

"""

df = pd.read_sql_query(query, conn)
df

,id_local,estado,bacia,campo
0,1,Rio de Janeiro,Campos,MARLIM LESTE
1,2,Rio de Janeiro,Santos,BÚZIOS
2,3,Rio de Janeiro,Santos,TAMBUATÁ
3,4,Rio de Janeiro,Santos,MERO
4,5,Rio de Janeiro,Santos,LULA
5,6,Rio de Janeiro,Santos,SUL DE LULA
6,7,Rio de Janeiro,Santos,AnC_LULA
7,8,São Paulo,Santos,SAPINHOÁ
8,9,São Paulo,Santos,SUDOESTE DE SAPINHOÁ
9,10,São Paulo,Santos,NOROESTE DE SAPINHOA


# Pocos - TODO

*   Incompleto: problema do poço em vários campos




In [ ]:
# INSERT INTO
#   Pocos(nome_poco_anp, id_local)


cmd = """
SELECT DISTINCT
  O.nome_poco_anp, L.estado, L.bacia, L.campo, L.id_local
FROM 
  OriginalDataSet as O
INNER JOIN 
  Locais as L
ON
  O.campo = L.campo AND O.bacia = L.bacia AND O.estado = L.estado
"""

df = pd.read_sql_query(cmd, conn)
df

# conn.execute(cmd)